In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'D:\documents\internship_project\intents.json').read()
intents = json.loads(data_file)

In [2]:
from nltk.tokenize import word_tokenize
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
#from
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

26 documents
6 classes ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
48 unique lemmatized words ['a', 'age', 'am', 'anyone', 'are', 'buy', 'call', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodbye', 'guy', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'later', 'leaving', 'like', 'menu', 'name', 'of', 'old', 'on', 'open', 'operation', 'reccommend', 'see', 'should', 'something', 'the', 'there', 'tim', 'to', 'up', 'what', 'whats', 'when', 'you', 'your']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\VAISHN~1\AppData\Local\Temp/ipykernel_1780/2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

c:\Users\VAISHNAVI\anaconda3\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 1.8710 - accuracy: 0.1538
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8167 - accuracy: 0.2308
Epoch 3/200
6/6 [==============================] - 0s 1ms/step - loss: 1.8071 - accuracy: 0.1923
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6910 - accuracy: 0.3462
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7075 - accuracy: 0.2308
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5990 - accuracy: 0.3462
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4961 - accuracy: 0.4615
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4865 - accuracy: 0.5000
Epoch 9/200
6/6 [==============================] - 0s 1ms/step - loss: 1.3737 - accuracy: 0.5769
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3166 - accuracy: 0.5769
Epoch 11/200
6/6 [===========